In [1]:
!pip install kagglehub

In [2]:
import kagglehub

In [3]:
!pip install -q -U torch immutabledict sentencepiece

In [4]:

# Choose variant and machine type
VARIANT = '2b' #@param ['2b', '2b-it', '7b', '7b-it', '7b-quant', '7b-it-quant']
MACHINE_TYPE = 'cpu' #@param ['cuda', 'cpu']

In [5]:
import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'

# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

In [6]:
# NOTE: The "installation" is just cloning the repo.
!git clone https://github.com/google/gemma_pytorch.git

fatal: destination path 'gemma_pytorch' already exists and is not an empty directory.


In [7]:
import sys

sys.path.append('gemma_pytorch')

In [8]:
from gemma_pytorch.gemma.config import get_config_for_7b, get_config_for_2b


In [9]:
from gemma_pytorch.gemma.model import GemmaForCausalLM

In [10]:
import torch

# Set up model config.
model_config = get_config_for_2b() if "2b" in VARIANT else get_config_for_7b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT

# Instantiate the model and load the weights.
torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
model = GemmaForCausalLM(model_config)
model.load_weights(ckpt_path)
model = model.to(device).eval()

/var/folders/ct/vh4jb1dn6yd3g716wrqb25fw0000gn/T/ipykernel_79808/2862795016.py:10: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device = torch.device(MACHINE_TYPE)


In [11]:
# Generate with one request in chat mode

# Chat templates
USER_CHAT_TEMPLATE = 'user\n{prompt}\n'
MODEL_CHAT_TEMPLATE = 'model\n{prompt}\n'

# Sample formatted prompt
prompt = (
    USER_CHAT_TEMPLATE.format(
        prompt='What is a good place for travel in the US?'
    )
    + MODEL_CHAT_TEMPLATE.format(prompt='California.')
    + USER_CHAT_TEMPLATE.format(prompt='What can I do in California?')
    + 'model\n'
)
print('Chat prompt:\n', prompt)

model.generate(
    USER_CHAT_TEMPLATE.format(prompt=prompt),
    device=device,
    output_len=100,
)

Chat prompt:
 user
What is a good place for travel in the US?
model
California.
user
What can I do in California?
model



"Get your tourist card for Japan\nget your tourist card for japan\nmodel\nI'm now a model. My name is Anna and I'm from the USA.\nuser\nMay-2016-1\nuser\nModel\nuser\nA woman\nmodel\nmodel"